In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import neurd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/usr/local/lib/python3.8/dist-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/usr/local/lib/python3.8/dist-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/usr/local/lib/python3.8/dist-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argument 

In [4]:
from mesh_tools import trimesh_utils as tu
from python_tools import ipyvolume_utils as ipvu
from neurd import neuron_visualizations as nviz

In [5]:
neurd.set_volume_params(
    volume = "microns"
)

# Data Prep: Loading mesh

In [ ]:
from mesh_tools import trimesh_utils as tu

In [ ]:
segment_id = 864691135510518224

In [ ]:
mesh_filepath = f"./{segment_id}.off"
mesh_filepath = '864691135510518224_small_double_soma.off'
mesh = tu.load_mesh_no_processing(mesh_filepath)
mesh

# Step 1: Decimation

In [ ]:
#/mnt/lab/users/celiib/neurd_packages/neuron_mesh_tools/Auto_Proofreading/Minnie65_Pipeline/Morphology/m65_morphology_2_Decimation_v1.ipynb

In [ ]:
decimation_kwargs = dict(
    decimation_ratio =0.25,
)

In [ ]:
mesh_dec = tu.decimate(
    mesh,
    **decimation_kwargs
)

In [ ]:
print(f"mesh = {mesh}")
print(f"mesh_dec = {mesh_dec}")

In [ ]:
ipvu.plot_objects(
    meshes = [mesh,mesh_dec],
    meshes_colors=["green","red"],
)

# Step 2: Some Identification

In [ ]:
#/mnt/lab/users/celiib/neurd_packages/neuron_mesh_tools/Auto_Proofreading/Minnie65_Pipeline/Morphology/m65_morphology_3_MeshFragment_full_v1.ipynb

In [ ]:
from neurd import soma_extraction_utils as sm
sm.output_global_parameters_nuclei()

In [ ]:
mesh_dec = mesh

In [ ]:
(total_soma_list, 
 run_time, 
 total_soma_list_sdf,
 glia_pieces,
 nuclei_pieces) = sm.extract_soma_center(
    segment_id,
    mesh = mesh_dec,
    return_glia_nuclei_pieces=True,
    verbose = True,
)

In [ ]:
from neurd import neuron_visualizations as nviz
nviz.plot_soma_extraction_meshes(
    mesh_dec,
    total_soma_list,
    glia_pieces,
    nuclei_pieces,
    verbose = True,
)

# Step 3: Decomposition

In [ ]:
from neurd import neuron

In [ ]:
#/mnt/lab/users/celiib/neurd_packages/neuron_mesh_tools/Auto_Proofreading/Minnie65_Pipeline/Auto_Proof/m65_autoproof_4_Decomposition_full_v1.ipynb

In [ ]:
neurd.set_volume_params(
    volume = "microns"
)

In [ ]:
somas = [
 total_soma_list, 
 run_time, 
 total_soma_list_sdf,
]

In [ ]:
neuron_obj = neuron.Neuron(
    mesh = mesh,
    somas = somas,
    suppress_preprocessing_print=False,
    
    suppress_output=False,
    glia_meshes=glia_pieces,
    nuclei_meshes = nuclei_pieces,
)

In [ ]:
nviz.visualize_neuron(
    neuron_obj,
    limb_branch_dict="all"
)

## calculating statistics about neuron

In [ ]:
from neurd.microns_volume_utils import data_interface as hdju
voxel_adjustment_vector = hdju.voxel_to_nm_scaling

stats_dict = neuron_obj.neuron_stats(stats_to_ignore = [
                    "n_boutons",
                     "axon_length",
                     "axon_area",
                     "max_soma_volume",
                     "max_soma_n_faces",],
    include_skeletal_stats = True,
    include_centroids= True,
    voxel_adjustment_vector=voxel_adjustment_vector,
)

stats_dict

## skeleton and skeleton stats

In [ ]:
skeleton = neuron_obj.skeleton

ipvu.plot_objects(
    main_skeleton=skeleton
)

In [ ]:
from neurd import neuron_statistics as nst

sk_stats = nst.skeleton_stats_from_neuron_obj(
    neuron_obj,
    include_centroids=True,
    voxel_adjustment_vector=voxel_adjustment_vector,
    verbose = True
)

sk_stats

# Step 3.5

In [ ]:
from neurd import neuron_utils as nru

filepath = nru.save_compressed_neuron(
    neuron_obj,
    output_folder = "./",
    file_name = f"large_double_soma",
    return_file_path = True,
)

# Step 4: Split Multi-Soma Merges

In [ ]:
#/mnt/lab/users/celiib/neurd_packages/neuron_mesh_tools/Auto_Proofreading/Minnie65_Pipeline/Auto_Proof/m65_autoproof_5_DecompositionSplitMultiSoma_full_v1.ipynb

In [ ]:
from neurd import proofreading_utils as pru

neurd.set_volume_params(
    "microns"
)

In [ ]:
plot_intermediates = True
plot_suggestions = True
plot_cut_coordinates = True
only_multi_soma_paths = False
verbose = True

In [ ]:
(split_results,
red_blue_split_results) = pru.multi_soma_split_suggestions(
    neuron_obj,
    plot_intermediates=plot_intermediates,
    plot_suggestions=plot_suggestions,
    plot_cut_coordinates = plot_cut_coordinates,
    only_multi_soma_paths=only_multi_soma_paths,
    default_cut_edge = "last",
    verbose = verbose,
    output_red_blue_suggestions = True,
    split_red_blue_by_common_upstream = True,
    apply_valid_upstream_branches_restriction = True,
    debug_red_blue = False,
)

In [ ]:
from neurd import soma_splitting_utils as ssu
ssu.plot_red_blue_split_suggestions_per_limb(
    neuron_obj,
    red_blue_split_results,
    
)

# Step 5: Splitting Multi-Neuron

In [ ]:
import numpy as np
from neurd import soma_splitting_utils as ssu
from neurd import proofreading_utils as pru

neurd.set_volume_params(
    "microns"
)

In [ ]:
n_paths_cut = pru.get_n_paths_cut(
    split_results,
    verbose = True)

In [ ]:
verbose = True

(neuron_list,
neuron_list_errored_limbs_area,
neuron_list_errored_limbs_skeletal_length,
neuron_list_n_multi_soma_errors,
neuron_list_n_same_soma_errors) = pru.split_neuron(neuron_obj,
    limb_results=split_results,
    verbose=verbose,
    return_error_info=True,
)

multiplicity = len(neuron_list)

# Cell Typing

In [ ]:
"""
Functions to be implemented in volume_utils

nuclei_from_segment_id
-> returns nucleus_ids,nucleus_centers

"""

In [ ]:
verbose = True

hdju = pru.data_mod
hdju

In [ ]:
split_index = 0
neuron_obj_pre_filt = neuron_list[split_index]

In [ ]:
# 1) Retrieving the corresponding nucleus ids
nucleus_ids,nucleus_centers = hdju.nuclei_from_segment_id(
    segment_id,
    return_centers=True,
    return_nm=True
    )

if verbose:
    if nucleus_ids is not None:
        print(f"Number of Corresponding Nuclei = {len(nucleus_ids)}")
        print(f"nucleus_ids = {nucleus_ids}")
        print(f"nucleus_centers = {nucleus_centers}")
        


In [ ]:
plot_initial_neuron = True

if plot_initial_neuron:
    neuron_obj_rot = hdju.align_neuron_obj(neuron_obj_pre_filt)
    nviz.visualize_neuron(neuron_obj_rot,limb_branch_dict = "all")

In [ ]:
nviz.visualize_neuron(neuron_obj_pre_filt,limb_branch_dict = "all")

In [ ]:
#2) Refining the width array to match sksleton coordinates if it doesn't

from neurd import branch_utils as bu

bu.refine_width_array_to_match_skeletal_coordinates(
        neuron_obj_pre_filt,
        verbose = False
)

In [ ]:
from neurd import neuron_simplification as nsimp

plot_floating_end_nodes_limb_branch_dict =  False
plot_downstream_path_limb_branch = False
plot_after_simplification = True

# 3) Performs branch simplification so there should always be 2 or more child branches
neuron_obj_pre_filt_after_simp = nsimp.branching_simplification(
    neuron_obj_pre_filt,
    return_copy = True,

    #floating endpiece arguments
    plot_floating_end_nodes_limb_branch_dict = plot_floating_end_nodes_limb_branch_dict,
    plot_final_neuron_floating_endpoints = False,

    # combine path arguments
    plot_downstream_path_limb_branch = plot_downstream_path_limb_branch,
    plot_final_neuron_path = False,
    plot_after_simplification = True,
    verbose = verbose,
)



In [ ]:
# Filter away a large cluster of dendrite if requested SKIPPED

filter_low_branch_cluster_dendrite = False
plot_limb_branch_filter_away_low_branch = False

if filter_low_branch_cluster_dendrite:
    neuron_obj_pre_class, filtering_info_low_branch = pru.apply_proofreading_filters_to_neuron(
        input_neuron = neuron_obj_pre_filt_after_simp,
        filter_list = [pru.low_branch_length_clusters_dendrite_filter],
        plot_limb_branch_filter_with_disconnect_effect=False,
        plot_limb_branch_filter_away=plot_limb_branch_filter_away_low_branch,
        plot_final_neuron=False,

        return_error_info=True,
        verbose=False,
        verbose_outline=verbose)
else:
    neuron_obj_pre_class = neuron_obj_pre_filt_after_simp
    filtering_info_low_branch = {}

In [ ]:
from neurd import neuron_utils as nru

winning_nucleus_id, nucleus_info = nru.pair_neuron_obj_to_nuclei(
    neuron_obj_pre_class,
    "S0",
    nucleus_ids,
    nucleus_centers,
    nuclei_distance_threshold = 15000,
    return_matching_info = True,
    verbose=True)

if verbose:
    print(f"nucleus_info = {nucleus_info}")
    print(f"winning_nucleus_id = {winning_nucleus_id}")


In [ ]:
# Getting the cell types from the database

database_cell_type_info = hdju.nuclei_classification_info_from_nucleus_id(
    winning_nucleus_id
)

database_e_i_class = database_cell_type_info[f"external_cell_type"]  

if verbose:
    print(f"database_cell_type_info = {database_cell_type_info}")
    print(f"database_e_i_class = {database_e_i_class}")

In [ ]:
# Adding the synapses to the cell
plot_synapses = True


neurd.set_volume_params(
    "microns"
)

from neurd import synapse_utils as syu
from neurd.microns_volume_utils import data_interface as hdju
hdju.set_synapse_filepath(synapse_filepath = '/neurd_packages/NEURD/Applications/autoproof_neuron_pipeline/864691135510518224_synapses.csv')

neuron_obj_attr = syu.add_synapses_to_neuron_obj(
    neuron_obj_pre_class,
    segment_id = segment_id,
    #synapse_filepath = '/neurd_packages/NEURD/Applications/autoproof_neuron_pipeline/864691135510518224_synapses.csv',
    validation = False,
    verbose  = verbose,
    original_mesh = mesh_dec,
    plot_valid_error_synapses = False,
    calculate_synapse_soma_distance = False,
    add_valid_synapses = True,
    add_error_synapses=False,
)

if plot_synapses:
    syu.plot_synapses(neuron_obj_attr)

In [ ]:
syu.plot_synapses(neuron_obj_attr)

In [ ]:
# Adding the spines
from neurd import spine_utils as spu

plot_spines = True

neuron_obj_attr = spu.add_head_neck_shaft_spine_objs(
    neuron_obj_attr,
    verbose = verbose
)

if plot_spines:
    spu.plot_spines_head_neck(neuron_obj_attr)

In [ ]:
# calculating the cell type

from neurd import cell_type_utils as ctu

plot_spines_and_sk_filter_for_syn = False
plot_spines_and_sk_filter_for_spine = False

baylor_e_i,baylor_cell_type_info = ctu.e_i_classification_from_neuron_obj(
    neuron_obj_attr,
    plot_on_model_map=False,
    plot_spines_and_sk_filter_for_syn = plot_spines_and_sk_filter_for_syn,
    plot_spines_and_sk_filter_for_spine = plot_spines_and_sk_filter_for_spine,
    verbose = verbose,
    return_cell_type_info = True
)

baylor_cell_type_info["baylor_cell_type"] = baylor_e_i 

if verbose:
    print(f"baylor_cell_type_info = \n{baylor_cell_type_info}")

In [ ]:
e_i_class = baylor_e_i

# Classifying axon and dendrite

In [ ]:
from neurd import axon_utils as au

neurd.set_volume_params(
    "microns"
)

filter_dendrite_on_axon = False
plot_initial_axon = False
plot_axon_on_dendrite = False
plot_high_fidelity_axon = False
plot_boutons_web = False



(o_neuron_unalign,
 filtering_info,
 axon_angles_dict,
 G_axon_labeled,)=au.complete_axon_processing(
            neuron_obj_attr,
            cell_type = e_i_class,
            add_synapses_and_head_neck_shaft_spines = False,
            validation = False,
            plot_initial_axon=plot_initial_axon,
            plot_axon_on_dendrite=plot_axon_on_dendrite,
            return_filtering_info = True,
             return_axon_angle_info = True,
            plot_high_fidelity_axon = plot_high_fidelity_axon,
            plot_boutons_web = plot_boutons_web,
            add_synapses_after_high_fidelity_axon = True,
            filter_dendrite_on_axon = filter_dendrite_on_axon,
            return_G_axon_labeled = True,
            verbose = verbose)

In [ ]:
ipvu.plot_objects(
    o_neuron_unalign.mesh,
    main_skeleton=o_neuron_unalign.axon_skeleton
)

# Proofreading

In [ ]:
import numpy as np
ipvu.plot_scatter(
    array=np.array([[0.12068656, 0.34903741, 0.84620654],
       [0.04812803, 0.14411268, 0.54990798],
       [0.79566505, 0.63533888, 0.03276685],
       [0.55378168, 0.25424957, 0.71898828],
       [0.1054361 , 0.70734158, 0.25456412],
       [0.51331846, 0.46793084, 0.44032635],
       [0.58216843, 0.46214255, 0.12553144],
       [0.08834693, 0.13477684, 0.88986768],
       [0.39226641, 0.96131601, 0.05423914],
       [0.86158737, 0.30404951, 0.42807901]]),
    color=(1,0,0,0.99),
    size=10,
    widgets_to_plot = ("size","marker","color"),
)

In [ ]:
size = [0.3]
color = [[1.0, 0.0, 0.0, 0.5]]
scatters = [np.array([[0.12068656, 0.34903741, 0.84620654],
       [0.04812803, 0.14411268, 0.54990798],
       [0.79566505, 0.63533888, 0.03276685],
       [0.55378168, 0.25424957, 0.71898828],
       [0.1054361 , 0.70734158, 0.25456412],
       [0.51331846, 0.46793084, 0.44032635],
       [0.58216843, 0.46214255, 0.12553144],
       [0.08834693, 0.13477684, 0.88986768],
       [0.39226641, 0.96131601, 0.05423914],
       [0.86158737, 0.30404951, 0.42807901]])]

ipvu.plot_multi_scatters(
    scatters,
    color="red",
    #size = size
)

In [ ]:
ipvu.plot_objects(
    skeletons=[np.array([[1,2,3],[4,5,6]]).reshape(-1,2,3)],
    scatters=[np.random.rand(10,3)]
)

In [ ]:
from python_tools import ipyvolume_utils as ipvu
import ipyvolume  as ipv

ipv.clear()
ipvu.plot_objects(
    scatters=[np.random.rand(10,3)],
    scatter_size=10,
    set_zoom=1,
    axis_box_off=False,
)

In [ ]:
# Proofreading

In [ ]:
len(dict_info["postsyn"]["synapse_ids"])

In [ ]:
from python_tools import pandas_utils as pu
df = syu.synapse_df_from_synapse_dict(dict_info,segment_id = segment_id)
filepath = f"{segment_id}_synapses.csv"
pu.df_to_csv(df,filepath)

In [ ]:
df

In [ ]:
df

In [ ]:
syu.synapse_dict_from_synapse_csv(
    synapse_filepath = '/neurd_packages/NEURD/Applications/autoproof_neuron_pipeline/864691135510518224_synapses.csv',
)

In [ ]:
synapse_ids

In [ ]:
# find the overlap in files in meshAfterParty
from pathlib import Path
from python_tools import file_utils as filu
from python_tools import pathlib_utils as plu
import numpy as np
map_files = plu.files_of_ext_type(
    "/neurd_packages/meshAfterParty/meshAfterParty/",
    ext = "py",
    return_stem = True,
)

neurd_files = plu.files_of_ext_type(
    "/neurd_packages/NEURD/neurd/",
    ext = "py",
    return_stem = True,
)

interset_files = np.intersect1d(map_files,neurd_files)
interset_files

In [ ]:
hdju_min.fetch_soma_mesh

In [ ]:
"""
Purpose: Want to move the overlapping files 
in meshAfterParty into a saved folder with a new name

Pseudocode: 
0) Create a new folder to save files in
1) cp the file from meshAfterParty to new folder (with suffix)
2) Delete the old filepath
"""
from python_tools import system_utils as su

new_folder = "/neurd_packages/meshAfterParty/meshAfterParty/neurd_moved_files"
existing_path = "/neurd_packages/meshAfterParty/meshAfterParty/"
suffix = "_neurd"
delete_old_files = True
verbose = True

Path(new_folder).mkdir(exist_ok=True)

for f in interset_files:
    curr_path = Path(existing_path) / Path(f"{f}.py")
    final_path =  Path(new_folder) / Path(f"{f}{suffix}.py")
    if curr_path.exists():
        su.copy_file(
           curr_path,
           final_path,
        )
        
        if verbose:
            print(f"Moved {curr_path}")

        if delete_old_files:
            curr_path.unlink()

In [ ]:
from os import sys
sys.path.append("/neurd_packages/meshAfterParty/meshAfterParty/")

In [ ]:
from dataInterfaceMinnie65 import data_interface as hdju_min

In [ ]:
hdju.nuclei_classification_info_from_nucleus_id(
    4078
)